# Classifying/Predicting  Molecular Subtypes in Breast Cancer Tumors 



# Part 3: Loading Packages and Reading in datasets in Rstudio

library(stringr) #assist with text manipulation
library(dplyr) # data manipulation
library(readr) # data input
library(caret) #select tuning parameters
library(MASS) # contains the data
library(nnet) # used for Multinomial Classification 
library(readr) #assist with text manipulation
library(kernlab) #assist with SVM feature selection
library(class) # used for an object-oriented style of programmin
library(KernelKnn) # used for K- Nearest-Neighbors method
library(nnet) # Used for Neural Net
library(e1071) 
library(gbm)
library(xgboost) # Used for xgbTree

In [ ]:
SSC <-read_csv("../input/transposed/77_cancer_proteomes_CPTAC_itraq.csv")
dim(SSC) #86 records, 12553 predictors
clin <-read_csv("../input/editpam50/clinical_data_breast_cancer.csv")
dim(clin) #105 Records, 30 clinical predictors/measurements

# Part 4: Using Stringr to Manipulate Complete TCGA ID
 
 
 The code below manipulates the Complete TCGA ID number in the Proteomes dataset, queries the Complete TCGA ID number and Molecular tumor type from the Clinical dataset and combines the Proteomes and Clinical dataset as **MASTER**.

In [ ]:
p1<-str_replace(SSC$`Complete TCGA ID`,"TCGA","")
p2<-str_replace(p1,".\\d+$","")
p3<-str_replace(p2,"^","TCGA-")

for (i in seq(1:83)){
  SSC$`Complete TCGA ID`[i]<-p3[i]
}

colnames(clin)
clinp<-clin[,c(1,21)]

MASTER<- inner_join(SSC,clinp)

# Part 5: Using Only Variables/Predictors With At Least 99% Of The Data



In [ ]:
#use only variables with 99% of the data
pcent<-sapply(MASTER, function(x) sum(!is.na(x))/length(x))
              
clean <- MASTER[1]
for (i in seq(2, 12554))
{
  if(pcent[i]>0.99){
    clean<-cbind(clean,MASTER[i])
  }
}

# Remove any NA's in the data frame,change to 0
clean[is.na(clean)] <- 0
CDF<-clean

#Remove the Complete TCGA ID from the CDF data frame.
# No longer needer for molecular classification
CDF<-CDF[-1]
attach(CDF)


The results of the Lasso data reduction technique are too large to be view in the Rstudio console. Therefore the results exported into a txt file on your local machine. 

In [ ]:
## Randomly dividing our dataset, which encompasses 4 major molecular tumor types, into datasets for testing and training out model. 
library(glmnet)
CDF$`PAM50 mRNA`<- as.factor(CDF$`PAM50 mRNA`)
x<-model.matrix(CDF$`PAM50 mRNA`~., CDF)
train<-sample(1:nrow(x), nrow(x)/2)
test<-(-train)
y<-CDF$`PAM50 mRNA`
y.test<-y[test]
set.seed(1) 


#Setting up parameters 
grid <- 10^seq(10,-2, length =100)
CDF$`PAM50 mRNA`<- as.factor(CDF$`PAM50 mRNA`)
set.seed (1)
cv.out<-cv.glmnet(x[train,],y[train],alpha=1,lambda =grid,family="multinomial",type.multinomial="grouped")
plot(cv.out)
bestlam<-cv.out$lambda.min

#  The results of the Lasso data reduction technique are exported onto your local machine for optimal viewing.
out<-glmnet(x,y,alpha =1, lambda=grid,family="multinomial",type.multinomial="grouped")
lasso.coef<-predict(out,type ="coefficients",s=bestlam,family="multinomial",type.multinomial="grouped")
sink(file="lasso.txt")
options("max.print"=8020)
lasso.coef
sink(NULL)

#Reading in and printing the column numbers of all genes for final dataset indexing
sink(file="colindex.txt")
options("max.print"=8100)
colnames(CDF)
sink(NULL)

# List of the 56 genetic predictors and the assoicated column number

`myoferlin isoform a` 142
`heat shock protein HSP 90-beta isoform a` 258
`keratin, type II cytoskeletal 72 isoform 1` 283
`cingulin` 342
`dedicator of cytokinesis protein 1` 537
`keratin, type I cytoskeletal 23`774
`1-phosphatidylinositol 4,5-bisphosphate phosphodiesterase beta-3 isoform 1` 1017
`TBC1 domain family member 1 isoform 2` 1069
`kinesin-like protein KIF21A isoform 1` 1107
`ubiquitin carboxyl-terminal hydrolase 4 isoform a` 1125
`PREDICTED: myomegalin-like` 1164
`KN motif and ankyrin repeat domain-containing protein 1 isoform a` 1280
`spermatogenesis-associated protein 5` 1281
`TBC1 domain family member 9` 1352
`receptor tyrosine-protein kinase erbB-2 isoform a precursor` 1376
`epidermal growth factor receptor isoform a precursor` 1377
`UPF0505 protein C16orf62` 1572
`MICAL-like protein 1` 1643
`UTP--glucose-1-phosphate uridylyltransferase isoform a` 1717
`probable ATP-dependent RNA helicase DDX6` 1848
`L-lactate dehydrogenase B chain` 1916
`myelin expression factor 2` 2037
`histone-lysine N-methyltransferase NSD3 isoform long` 2065
`histone-lysine N-methyltransferase NSD3 isoform short` 2067
`signal transducer and activator of transcription 6 isoform 1` 2540
`WD repeat-containing protein 91` 2726
`UPF0553 protein C9orf64` 3201
`ran-binding protein 9` 3226
`arfaptin-1 isoform 2` 3438
`tonsoku-like protein` 3567
`protein LSM14 homolog B` 3606
`oxysterol-binding protein-related protein 2 isoform 2` 3629
`PDZ domain-containing protein GIPC2` 3807
`calpain small subunit 2` 4075
`condensin-2 complex subunit G2` 4083
`telomere length regulation protein TEL2 homolog` 4088
`guanine nucleotide-binding protein G(olf) subunit alpha isoform 1` 4116
`squalene synthase` 4613
`cathepsin B preproprotein` 4816
`5'-nucleotidase domain-containing protein 2 isoform 1` 4894
`COBW domain-containing protein 1 isoform 2` 4914
`transcription elongation factor A protein-like 5`5218
`DNA repair protein XRCC4 isoform 1` 5542
`transmembrane protein 132A isoform a precursor` 5574
`transcription factor AP-2-beta` 5628
`alpha-methylacyl-CoA racemase isoform 3` 5693
`trans-acting T-cell-specific transcription factor GATA-3 isoform 1` 6111
`uncharacterized protein C7orf43` 6179
`molybdopterin synthase catalytic subunit large subunit MOCS2B` 6533
`Golli-MBP isoform 1` 6581
`hepatocyte nuclear factor 3-gamma` 6769
`39S ribosomal protein L40, mitochondrial`7015
`migration and invasion enhancer 1`7319
`ubiquitin-conjugating enzyme E2 E3` 7331
`protein S100-A13`7357
`transcription initiation factor TFIID subunit 8`7425
`phosphoribosyltransferase domain-containing protein 1`7512
`polyadenylate-binding protein-interacting protein 2`7923

# Part 8: Selecting Predictors By Column Number 



In [ ]:
testdf<-CDF[,c(8018,142,258,283,537,774,1017,1069,1107,1125,1164,1280,1281,1352,
               1376,1377,1572,1643,1717,1848,1916,2037,2065,2067,2540,2726,3201,
               3226,3438,3567,3606,3629,3807,4075,4083,4088,4116,4613,4816,4894,
               4914,5218,5542,5574,5693,6111,6179,6533,6581,6769,7015,7319,
               7331,7357,7425,7512,7923)]

# Part 9: Classifying Molecular Tumor Type Using Ensemble, Multinomial, K Nearest Neighbors, Support Vector Machine, and Other Machine Learning Methods 

## Molecular Tumor Type Using Ensemble-Boosting Regression Tree 

Gradient boosting is a machine learning technique for regression and classification problems, the basis involves 3 broad elements. 

1. A loss function to be optimized.
2. A weak learner to make predictions.
3. An additive model to add weak learners to minimize the loss function.

Gradient boosting produces a prediction model in the form of an ensemble of weak prediction models. 


In [ ]:
testdf$`PAM50 mRNA`<- as.factor(testdf$`PAM50 mRNA`)

gbm.model<-gbm(`PAM50 mRNA`~., data=testdf, shrinkage=0.01, distribution = 'multinomial',
               cv.folds=5, n.trees=5000, verbose=F)
summary(gbm.model)
best.iter<-gbm.perf(gbm.model, method="cv")

In [ ]:
fitControl <- trainControl(method="cv", number=10, returnResamp = "all",classProbs = TRUE, summaryFunction = multiClassSummary)
AccuraciesGLM<- c(0,0,0)
for(i in seq(20)){
  train<- createDataPartition(testdf$`PAM50 mRNA`, p = .70, list = FALSE)
  trainDF<-testdf[train,]
  testDF<-testdf[-train,]
  trainDF$`PAM50 mRNA`<-as.factor(trainDF$`PAM50 mRNA`)
  testDF$`PAM50 mRNA`<-as.factor(testDF$`PAM50 mRNA`)
  gbm.model<-gbm(`PAM50 mRNA`~., data=testdf, shrinkage=0.01, distribution = 'multinomial',
                 cv.folds=5, n.trees=5000, verbose=F)
  best.iter<-gbm.perf(gbm.model, method="cv")
  
  obj<-train(`PAM50 mRNA`~., data=trainDF, method="gbm",distribution="multinomial",
             trControl=fitControl, verbose=F,metric = "ROC",
             tuneGrid=data.frame(.n.trees=best.iter, .shrinkage=0.01, .interaction.depth=4, .n.minobsinnode=1))
  
  AccuraciesGLM[i] <- confusionMatrix(predict(obj,newdata=testDF[-1]),testDF$`PAM50 mRNA`)$overall["Accuracy"]
}

In [ ]:
plot(density(AccuraciesGLM))
summary(AccuraciesGLM)

## Molecular Tumor Type Using K- Nearest-Neighbors 


In [ ]:
library(class)
library(KernelKnn)
AccuraciesKNN<-c(0,0,0)
#choose which measurements to use in classification
tf<-testdf
x<-tf[2:57]
#choose which group labels to use in classification
y<-factor(testdf$`PAM50 mRNA`)

for (i in seq(300)){
  kcres3 <-knn.cv(x, y, k = 3, prob= TRUE)
  AccuraciesKNN[i] <-confusionMatrix(kcres3,testdf$`PAM50 mRNA`)$overall["Accuracy"]
}

In [ ]:
summary(AccuraciesKNN)
plot(density(AccuraciesKNN))

## Molecular Tumor Type Using Support Vector Machines



In [ ]:
library(kernlab)
AccuraciesSVM <- c(0,0,0)
for(i in seq(1000)){
  
  train<- createDataPartition(testdf$`PAM50 mRNA`, p = .70, list = FALSE)
  trainDF<-testdf[train,]
  testDF<-testdf[-train,]
  
  train.ksvm<-ksvm(`PAM50 mRNA` ~.,
                   scale = TRUE,
                   data=trainDF,
                   kernel="rbfdot",
                   prob.model=TRUE)
  
  AccuraciesSVM[i] <- confusionMatrix(testDF$`PAM50 mRNA`, predict(train.ksvm, testDF))$overall["Accuracy"]
}

In [ ]:
summary(AccuraciesSVM)
plot(density(AccuraciesSVM))

## Molecular Tumor Type Using Multinomial Method

The Multinomial method is a classifier that is used to predict the probabilities of the different possible outcomes of a categorically dependent variable. One assumption is that collinearity among independent variables are relatively low.

In [ ]:
AccuraciesMultinomial<- c(0,0,0)
for(i in seq(1000)){
  train<- createDataPartition(testdf$`PAM50 mRNA`, p = .70, list = FALSE)
  trainDF<-testdf[train,]
  testDF<-testdf[-train,]
  
  net<-multinom(`PAM50 mRNA`~.,
                data=trainDF,trace=FALSE)
  AccuraciesMultinomial[i] <- confusionMatrix(predict(net, newdata=testDF, "class"),testDF$`PAM50 mRNA`)$overall["Accuracy"]
}

In [ ]:
plot(density(AccuraciesMultinomial))
summary(AccuraciesMultinomial)

## Part 10: Conclusions 

Below are the mean results from the machine learning methods,  the models are ran mutlptile times so that one can compute the average the accuracy. 

### Support Vector Machines(SVM): 97%
### Neural Network:  96%
### Multinomial Method: 90%
### K- Nearest-Neighbors (KNN): 89%

### eXtreme Gradient Boosting with Random Variables: 55%

The highest mean classification accuracy resulted from the Support Vector Machines(SVM) and Neural Network model. Models that apply selected predictors from variable selection outperformed the eXtreme Gradient Boosting model with Random Variables in terms of classification accuracy. 

